In [ ]:
import pandas as pd

# Load the Excel file into a DataFrame
file_path = "Dataset/Vaxjo/VO ID term editing.xlsx"
df = pd.read_excel(file_path)

# Print the column names
print("Columns in the dataset:")
print(df.columns.tolist())


In [ ]:
import pandas as pd
import re
from Bio import Entrez

# Always set your email for NCBI API usage
Entrez.email = "your_email_here"

# Pick the column with sources
col = "definition source"



In [ ]:
import pandas as pd
import re
import requests

FILE_PATH = "Dataset/Vaxjo/VO ID term editing.xlsx"
COL = "definition source"
HTTP_TIMEOUT = 6

# ---------- Regexes ----------
PMID_EXPLICIT = [
    r"(?i)PMID[:\s]*([0-9]+)",
    r"(?i)Pubmed[:\s]*([0-9]+)",
    r"(?i)pubmed\.ncbi\.nlm\.nih\.gov/([0-9]+)",
    r"(?i)ncbi\.nlm\.nih\.gov/pubmed/\?term=([0-9]+)",
    r"(?i)[\?&]term=([0-9]+)",
]

NON_PUBMED_URL = re.compile(
    r"https?://(?!pubmed\.ncbi\.nlm\.nih\.gov|www\.ncbi\.nlm\.nih\.gov/pubmed)[^\s|,;]+",
    re.I,
)

BAN_PATTERNS = re.compile(
    r"\b(?:CHEBI|PMCID|PMC|NCT|ISSN|ISBN|DB|CAS|EC)\b[:#]?\s*\d+|"
    r"\bDOI[:\s]*10\.[^\s|,;]+",
    re.IGNORECASE
)


BARE_PMID = re.compile(r"\b([0-9]{6,9})\b")  # conservative: 6–9 digits

def extract_pmids_strict(text: str):
    if not text:
        return set()

    # 1) explicit sources
    explicit = set()
    for pat in PMID_EXPLICIT:
        explicit |= set(re.findall(pat, text))

    # 2) clean text for bare-number detection
    t = NON_PUBMED_URL.sub(" ", text)       # drop numbers inside non-PubMed URLs
    t = BAN_PATTERNS.sub(" ", t)            # remove obvious non-PMID contexts

    # if the cell has no PubMed cues and lots of words, be cautious with bare numbers
    has_pubmed_cue = bool(re.search(r"(?i)pmid|pubmed|ncbi", text))

    bare = set()
    if has_pubmed_cue:
        bare |= set(re.findall(BARE_PMID, t))
    else:
        # allow bare numbers only if the remaining text is mostly numbers/separators
        stripped = re.sub(r"[0-9,\s|;/\-]+", "", t)
        if stripped == "":
            bare |= set(re.findall(BARE_PMID, t))

    # normalize & dedupe
    return {str(int(x)) for x in (explicit | bare)}  # strip leading zeros, keep as strings

def verify_pmids(pmids, batch_size=200):
    """Verify via NCBI E-utilities (esummary)."""
    valid, invalid = set(), set()
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    headers = {"User-Agent": "pmid-verifier/1.1"}
    pmids = list(pmids)
    for i in range(0, len(pmids), batch_size):
        batch = pmids[i:i+batch_size]
        try:
            r = requests.get(base, params={"db":"pubmed","id":",".join(batch),"retmode":"json"},
                             headers=headers, timeout=HTTP_TIMEOUT)
            r.raise_for_status()
            data = r.json().get("result", {})
            uids = set(data.get("uids", []))
            for p in batch:
                (valid if p in uids and p in data else invalid).add(p)
        except Exception:
            invalid.update(batch)
    return valid, invalid


import requests

def verify_vac_links(links, timeout=6):
    """
    Returns sets of (valid, invalid).
    A VAC link is 'valid' if the server responds with 2xx or 3xx.
    """
    valid, invalid = set(), set()
    headers = {"User-Agent": "vac-link-checker/1.0"}
    for url in links:
        try:
            # try HEAD first
            r = requests.head(url, headers=headers, timeout=timeout, allow_redirects=True)
            if r.status_code >= 400:  # fallback to GET
                r = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
            if 200 <= r.status_code < 400:
                valid.add(url)
            else:
                invalid.add(url)
        except Exception:
            invalid.add(url)
    return valid, invalid



def extract_vac_links(text: str):
    if not text:
        return set()
    return set(re.findall(r"https?://vac\.niaid\.nih\.gov/view\?id=\d+", text))

# ---------- Run ----------
df = pd.read_excel(FILE_PATH)
series = df[COL].astype(str).where(df[COL].notna(), "")

empty_cells = 0
cells_with_pmid = 0
cells_with_vac = 0
cells_other = 0

all_pmids = set()
all_vac = set()

for cell in series:
    s = cell.strip()
    if not s:
        empty_cells += 1
        continue

    pmids_here = extract_pmids_strict(s)
    vac_here = extract_vac_links(s)

    if pmids_here:
        cells_with_pmid += 1
        all_pmids |= pmids_here
    if vac_here:
        cells_with_vac += 1
        all_vac |= vac_here
    if not pmids_here and not vac_here:
        cells_other += 1

valid_pmids, invalid_pmids = verify_pmids(all_pmids)

print("===== CELL CATEGORY STATS =====")
print(f"Total cells:        {len(series)}")
print(f"Empty:              {empty_cells}")
print(f"Contains PMID:      {cells_with_pmid}")
print(f"Contains VAC link:  {cells_with_vac}")
print(f"Other (non-empty):  {cells_other}")

avg_pmids_per_pmid_cell = (len(all_pmids) / cells_with_pmid) if cells_with_pmid else 0
print("\n===== PMID STATS =====")
print(f"Unique candidate PMIDs (strict): {len(all_pmids)}")
print(f"Valid PMIDs:                    {len(valid_pmids)}")
print(f"Invalid PMIDs:                  {len(invalid_pmids)}")
print(f"Avg PMIDs per PMID-cell:        {avg_pmids_per_pmid_cell:.2f}")

# Run check
valid_vac, invalid_vac = verify_vac_links(all_vac)

print("\n===== VAC LINK VALIDITY =====")
print(f"Total VAC links: {len(all_vac)}")
print(f"Valid VAC links: {len(valid_vac)}")
print(f"Invalid VAC links: {len(invalid_vac)}")
avg_vacs_per_vac_cell = (len(all_vac) / cells_with_vac) if cells_with_vac else 0
print(f"Avg VACs per VAC-cell: {avg_vacs_per_vac_cell:.2f}")


In [ ]:
len(all_pmids)

In [ ]:
# --- Inspect the 19 "Other (non-empty)" cells ---
other_cells = []

for idx, cell in series.items():
    s = cell.strip()
    if s and not extract_pmids_strict(s) and not extract_vac_links(s):
        other_cells.append((idx, s))

print(f"Found {len(other_cells)} 'Other' cells")

# Show first few for inspection
for i, (row, text) in enumerate(other_cells[:22], start=1):
    print(f"\n--- Other Cell #{i} (Row {row}) ---")
    print(text[:500])  # truncate long entries for readability


In [ ]:
import re
from openpyxl import load_workbook

vac_path = "Dataset/Vaxjo/VAC_September10_2025.xlsx"

# Regex + normalizer for VAC links
VAC_ID_RE = re.compile(r"https?://vac\.niaid\.nih\.gov/view\?id=(\d+)", re.I)
def normalize_vac(url: str):
    m = VAC_ID_RE.search(url or "")
    return f"https://vac.niaid.nih.gov/view?id={m.group(1)}" if m else None

# 1) Load workbook (first sheet), extract hyperlinks from column A
wb = load_workbook(vac_path, data_only=True, read_only=False)
ws = wb[wb.sheetnames[0]]

# Some hyperlinks are attached to cells, some live in ws.hyperlinks (range-based)
ref_to_target = {}
for hl in getattr(ws, "hyperlinks", []):
    if getattr(hl, "target", None) and getattr(hl, "ref", None):
        ref_to_target[hl.ref] = hl.target

found_vacs = set()
for r in range(1, ws.max_row + 1):
    url = None
    cell = ws[f"A{r}"]
    if cell.hyperlink and getattr(cell.hyperlink, "target", None):
        url = cell.hyperlink.target
    elif f"A{r}" in ref_to_target:
        url = ref_to_target[f"A{r}"]
    norm = normalize_vac(url)
    if norm:
        found_vacs.add(norm)

print(f"VAC links embedded in first column: {len(found_vacs)}")

# 2) Compare against your existing set `all_vac`
#    (assumes you already have `all_vac` as the 64 unique VAC URLs)
all_vac_norm = {normalize_vac(u) for u in all_vac if normalize_vac(u)}
overlap = all_vac_norm & found_vacs
missing = all_vac_norm - found_vacs
extras  = found_vacs - all_vac_norm

print(f"Overlap with our {len(all_vac_norm)} VACs: {len(overlap)}")
print(f"Missing (in our 64 but not in file): {len(missing)}")
print(f"Extras (in file but not in our 64):  {len(extras)}")

# (Optional) peek
print("Examples present:", sorted(list(overlap))[:10])
print("Examples missing:", sorted(list(missing))[:10])


In [ ]:
import pandas as pd
import re
from openpyxl import load_workbook

vac_file = "Dataset/Vaxjo/VAC_September10_2025.xlsx"

# --- helpers ---
VAC_ID_RE = re.compile(r"https?://vac\.niaid\.nih\.gov/view\?id=(\d+)", re.I)

def vac_id_from_url(url: str):
    if not url:
        return None
    m = VAC_ID_RE.search(url)
    return m.group(1) if m else None

PMID_PATTERNS = [
    r"(?i)PMID[:\s]*([0-9]+)",
    r"(?i)Pubmed[:\s]*([0-9]+)",
    r"(?i)pubmed\.ncbi\.nlm\.nih\.gov/([0-9]+)",
    r"(?i)[\?&]term=([0-9]+)",
    r"\b([0-9]{5,9})\b",  # bare pmids (len 5–9)
]
def extract_pmids_cell(val) -> set:
    if pd.isna(val):
        return set()
    text = str(val)
    out = []
    for pat in PMID_PATTERNS:
        out += re.findall(pat, text)
    # normalize (strip leading zeros), dedupe
    return {str(int(x)) for x in out if x.isdigit()}

# --- read the sheet with pandas ---
df_vac = pd.read_excel(vac_file)  # first sheet by default

# --- read embedded hyperlinks from column A (first column) with openpyxl ---
wb = load_workbook(vac_file, data_only=True, read_only=False)
ws = wb[wb.sheetnames[0]]

# map range hyperlinks too (sometimes hyperlinks are stored in ws.hyperlinks)
ref_to_target = {}
for hl in getattr(ws, "hyperlinks", []):
    if getattr(hl, "target", None) and getattr(hl, "ref", None):
        ref_to_target[hl.ref] = hl.target

vac_urls = []
vac_ids  = []
start_row = 2  # row 1 is header; pandas row 0 == Excel row 2
for i in range(len(df_vac)):
    addr = f"A{start_row + i}"
    url = None
    cell = ws[addr]
    if cell.hyperlink and getattr(cell.hyperlink, "target", None):
        url = cell.hyperlink.target
    elif addr in ref_to_target:
        url = ref_to_target[addr]
    vac_urls.append(url)
    vac_ids.append(vac_id_from_url(url))

df_vac["VAC_URL"] = vac_urls
df_vac["VAC_ID"]  = vac_ids

# --- normalize your existing all_vac set to VAC_IDs ---
# assumes you already have `all_vac` (the 64 VAC URLs) from earlier code
all_vac_ids = {vac_id_from_url(u) for u in all_vac if vac_id_from_url(u)}

# --- filter to rows that match our all_vac ---
df_match = df_vac[df_vac["VAC_ID"].isin(all_vac_ids)].copy()

# --- keep the requested columns (rename if your headers differ) ---
wanted_cols = ["VAC_ID", "Preclinical PubMedID", "Clinical Trial PubMedID"]
vac_pmid_df = df_match[wanted_cols].copy()

# --- build a unique PMID list from the two PMID columns ---
unique_pmids = set()
for col in ["Preclinical PubMedID", "Clinical Trial PubMedID"]:
    if col in vac_pmid_df.columns:
        for v in vac_pmid_df[col]:
            unique_pmids |= extract_pmids_cell(v)

unique_pmids = sorted(unique_pmids, key=int)

print(f"Rows matched to our VAC set: {len(vac_pmid_df)}")
print(f"Unique PMIDs collected: {len(unique_pmids)}")
# peek
print(len(unique_pmids))


In [ ]:
# Merge all_pmids (from your main parse) + unique_pmids (from VAC sheet)
def _norm_ids(xs):
    return {str(int(x)) for x in xs if str(x).strip().isdigit()}

ultimate_pmids = sorted(_norm_ids(all_pmids) | _norm_ids(unique_pmids), key=int)

print(f"Ultimate unique PMIDs: {len(ultimate_pmids)}")
print(len(ultimate_pmids))  # peek

# (optional) DataFrame / export
# import pandas as pd
# pd.DataFrame({"PMID": ultimate_pmids}).to_csv("ultimate_pmids.csv", index=False)


In [ ]:
import requests


from bs4 import BeautifulSoup
import time

# ---------------------
# 1. Fetch PubMed (title + abstract)
# ---------------------
def fetch_pubmed_info(pmids, batch_size=100, email="your.email@example.com"):
    """
    Fetch title + abstract for a list of PMIDs.
    Returns dict: PMID -> {title, abstract}
    """
    results = {}
    base = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    headers = {"User-Agent": "pubmed-fetcher/1.0"}
    for i in range(0, len(pmids), batch_size):
        batch = pmids[i:i+batch_size]
        params = {
            "db": "pubmed",
            "id": ",".join(batch),
            "rettype": "abstract",
            "retmode": "xml",
            "email": email
        }
        r = requests.get(base, params=params, headers=headers, timeout=20)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")
        for article in soup.find_all("pubmedarticle"):
            pmid = article.pmid.text
            title = article.articletitle.text if article.articletitle else ""
            abstract = " ".join([ab.text for ab in article.find_all("abstracttext")])
            results[pmid] = {"title": title, "abstract": abstract}
        time.sleep(0.4)  # throttle to be kind to NCBI
    return results


# ---------------------
# 2. Fetch VAC entry page
# ---------------------
def fetch_vac_info(vac_links):
    """
    Fetch page <title> + main text for VAC entries.
    Returns dict: VAC_URL -> {title, text}
    """
    results = {}
    headers = {"User-Agent": "vac-fetcher/1.0"}
    for url in vac_links:
        try:
            r = requests.get(url, headers=headers, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "lxml")
            title = soup.title.string if soup.title else ""
            text = " ".join(p.get_text(strip=True) for p in soup.find_all("p"))
            results[url] = {"title": title, "text": text[:5000]}  # limit text size
        except Exception as e:
            results[url] = {"title": "", "text": f"ERROR: {e}"}
    return results


# ---------------------
# 3. Fetch Other links
# ---------------------
def fetch_other_info(other_links):
    """
    Generic scraper: title + paragraph text.
    Returns dict: URL -> {title, text}
    """
    results = {}
    headers = {"User-Agent": "generic-scraper/1.0"}
    for url in other_links:
        try:
            r = requests.get(url, headers=headers, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "lxml")
            title = soup.title.string if soup.title else ""
            text = " ".join(p.get_text(strip=True) for p in soup.find_all("p"))
            results[url] = {"title": title, "text": text[:5000]}
        except Exception as e:
            results[url] = {"title": "", "text": f"ERROR: {e}"}
    return results


# ---------------------
# Example usage (assuming you already have sets all_pmids, all_vac, other_links)
# ---------------------
pmid_info = fetch_pubmed_info(sorted(ultimate_pmids))


In [ ]:
# assume pmid_info = fetch_pubmed_info(ultimate_pmids)

empty_abstracts = [pid for pid, rec in pmid_info.items() if not rec.get("abstract")]
print(f"Total PMIDs fetched: {len(pmid_info)}")
print(f"Abstracts missing/empty: {len(empty_abstracts)}")

# peek at first few
print("Examples:", empty_abstracts)


In [ ]:
import json

out_path = "Dataset/Vaxjo/All PMID abstracts.txt"
with open(out_path, "w", encoding="utf-8") as f:
    # compact JSON: no extra spaces or newlines
    json.dump(pmid_info, f, ensure_ascii=False, separators=(",", ":"))

print(f"Wrote: {out_path}")


In [ ]:
print(type(ultimate_pmids))
print(isinstance(ultimate_pmids, list))


In [ ]:
all_strings = all(isinstance(pmid, str) for pmid in ultimate_pmids)
print(all_strings)  # True means all elements are strings


In [ ]:
%pip install pmc-id-converter

In [ ]:
import requests

pmid_list = ultimate_pmids

import requests

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

api_url = 'https://pmc.ncbi.nlm.nih.gov/tools/idconv/api/v1/articles/'

# Add your email and tool name here
tool_name = 'my_tool'

email = "your_email_here"


all_results = []

for batch in chunks(pmid_list, 200):
    params = {
        'ids': ','.join(batch),
        'idtype': 'pmid',
        'format': 'json',
        'tool': tool_name,
        'email': email
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    all_results.extend(data.get('records', []))

# Now all_results contains PMCID mappings for all PMIDs
for record in all_results:
    print(f"PMID: {record.get('pmid')}, PMCID: {record.get('pmcid')}")



In [ ]:
# Create DataFrame with selected columns
df = pd.DataFrame(all_results)[['pmid', 'pmcid']]

# Rename columns if desired (optional)
df.rename(columns={'pmid': 'PMID', 'pmcid': 'PMCID'}, inplace=True)

# Display the dataframe
print(df.head(4))

# Count how many PMCID are None or empty
none_count = df['PMCID'].isna().sum()
print(f"Number of PMIDs with PMCID: {len(all_results)-none_count}")
print(f"Number of PMIDs with no PMCID: {none_count}")

In [ ]:
import os
import requests
import time

os.makedirs('plaintext', exist_ok=True)
os.makedirs('xml', exist_ok=True)

base_url_xml = 'https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi'

failed_xml = []
failed_plaintext = []

def download_plaintext(pmcid):
    try:
        url = f'https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/'
        r = requests.get(url)
        if r.status_code == 200:
            return r.text
        else:
            print(f"Failed to download plaintext for {pmcid}: HTTP {r.status_code}")
            failed_plaintext.append(pmcid)
    except Exception as e:
        print(f"Error downloading plaintext for {pmcid}: {e}")
        failed_plaintext.append(pmcid)
    return None

def download_xml(pmcid):
    article_id = pmcid.replace('PMC', '')
    params = {
        'verb': 'GetRecord',
        'metadataPrefix': 'pmc',
        'identifier': f'oai:pubmedcentral.nih.gov:{article_id}'
    }
    try:
        r = requests.get(base_url_xml, params=params)
        if r.status_code == 200:
            return r.text
        else:
            print(f"Failed to download xml for {pmcid}: HTTP {r.status_code}")
            failed_xml.append(pmcid)
    except Exception as e:
        print(f"Error downloading xml for {pmcid}: {e}")
        failed_xml.append(pmcid)
    return None

for pmcid in df['PMCID'].dropna().unique():
    print(f"Downloading {pmcid} ...")

    xml_content = download_xml(pmcid)
    if xml_content:
        with open(f'Dataset/Vaxjo/xml/{pmcid}.xml', 'w', encoding='utf-8') as f_xml:
            f_xml.write(xml_content)

    plaintext_content = download_plaintext(pmcid)
    if plaintext_content:
        with open(f'Dataset/Vaxjo/plaintext/{pmcid}.txt', 'w', encoding='utf-8') as f_txt:
            f_txt.write(plaintext_content)

    time.sleep(1)

#print(f"\nFailed XML downloads ({len(failed_xml)}): {failed_xml}")
#print(f"Failed plaintext downloads ({len(failed_plaintext)}): {failed_plaintext}")


In [ ]:
# Save failed XML download IDs to a file
with open('Dataset/Vaxjo/failed_xml_downloads.txt', 'w', encoding='utf-8') as f_xml_fail:
    for pmcid in failed_xml:
        f_xml_fail.write(pmcid + '\n')

# Save failed plaintext download IDs to a file
with open('Dataset/Vaxjo/failed_plaintext_downloads.txt', 'w', encoding='utf-8') as f_txt_fail:
    for pmcid in failed_plaintext:
        f_txt_fail.write(pmcid + '\n')
